In [ ]:
def asp_try(asp_file, instance):
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:       
            with ctl.solve(yield_=True, async_=True) as handle: 
                handle.wait(10)
                handle.cancel()
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    #for m in models:
                    #    print(m.symbols(True,True,True))
            if ctl.statistics['solving']['conflicts'] > 0:
                print("Errors occurred during solving.")
                # Analyze the conflicts to get more information
                conflicts_analyzed = ctl.statistics['solving']['conflicts_analyzed']
                print("Conflicts analyzed: {}".format(conflicts_analyzed))
            else:
                print("No errors occurred during solving.")
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [ ]:
def asp_try_v2(asp_file, instance):
    
    input_files = [asp_file, instance]
    asp_program = []

    def custom_logger(code, message):
        if code == clingo.MessageCode.Warning:
            print(f"Warning: {message}")
        elif code == clingo.MessageCode.Error:
            print(f"Error: {message}")
        elif code == clingo.MessageCode.Info:
            print(f"Info: {message}")
        elif code == clingo.MessageCode.Debug:
            print(f"Debug: {message}")

    # Create a Control object with the custom logger
    control = clingo.Control(logger=custom_logger,message_limit=100)
    print("debug")
    def on_model(model):
        print("Model:", model)
        
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    print("debug2")
    
    control.ground([("base", [])])
    print("debug3")
    #except Exception as e:
    #    error_message = control.messages
    #    print(error_message)
    #    print(e)
    #    print("Grounding Error")
    #    return None
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
        print("No errors occurred during solving.")
    except Exception as e:
        print(e)
        #if control.statistics['solving']['solvers']['conflicts'] > 0:
        print("Errors occurred during solving.")
        # Analyze the conflicts to get more information
        conflicts_analyzed = control.statistics['solving'] #['conflicts_analyzed']
        print("Conflicts analyzed: {}".format(conflicts_analyzed))
        #else:            

In [ ]:
def asp_try_3(asp_file,instance):
    errors = []
    messages = []
    def custom_logger(code, message):
        errors.append(code)
        messages.append(message)
    control = clingo.Control(logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]
    asp_program = []
    models = []
    def on_model(model):
        print("Model:", model)
        models.append(model)
    
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    try:
        control.ground([("base", [])])
    except:
        print("Grounding stopped because of errors")
        error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
        sub_codes = ["unsafe"]
        error_dict = {}
        for error_code in error_codes:
            error_dict[error_code] = 0
        sub_dict = {}
        for sub_code in sub_codes:
            sub_dict[sub_code] = 0
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    sub_dict[sc] += 1
        return models, error_dict, message_dict
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    control.solve(on_model=on_model)
    
    return models, error_dict, message_dict